In [12]:
import dependencies
import line_profiler
import numpy as np

from scipy.stats import invwishart, wishart
from Scripts.utilities import LASSO, generate_confusion_matrices
from Scripts.utilities import precision, recall, accuracy
from Scripts.utilities import scale_diagonals_to_1, tr_p
from Scripts.utilities import kron_sum, kron_sum_diag, kron_prod
from Scripts.generate_data import generate_Ys
from Scripts.niBiGLasso import niBiGLasso

np.set_printoptions(precision=3, suppress=True)
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
Psi_gen, Theta_gen, Ys = generate_Ys(
    m=100, # Note that we only need m=3 for good vindication, whereas
    n=(n:=51), # m needs to be much larger to get good estimates in general
    p=(p:=49), # anyways (in this alg and scalable version).
    expected_nonzero_psi=n**2 / 5, # (divide by 5 for sparsity)
    expected_nonzero_theta=p**2 / 5,
    structure="Kronecker Sum",
    posdef_distr=invwishart#invwishart
)
print(Ys.shape)
print(Psi_gen.shape)
print(Theta_gen.shape)

(100, 51, 49)
(51, 51)
(49, 49)


In [103]:
def analyticBiGLasso(
    Ys: "(m, n, p) batch of m samples of (n, p) matrices",
    beta_1: "L1 penalty for within-row precision matrix",
    beta_2: "L2 penalty for within-column precision matrix",
    true_U = None,
    true_V = None
) -> (
    "(n, n) within-row precision matrix Psi",
    "(p, p) within-row precision matrix Theta"
):
    """
    
    """
    
    (m, n, p) = Ys.shape
    
    # Step one:
    # Calculate the empirical covariance matrices
    # Uncomment to get unbiased versions
    correction = m #m - 1 if m > 0 else m
    T_psi: "(n, n)" = np.einsum("mnp, mlp -> nl", Ys, Ys) / (correction * n)
    T_theta: "(p, p)" = np.einsum("mnp, mnl -> pl", Ys, Ys) / (correction * p)
        
    #return np.linalg.inv(T_psi), np.linalg.inv(T_theta), (T_psi, T_theta)
        
    assert T_psi.shape == (n, n)
    assert T_theta.shape == (p, p)
    
    # Step two:
    # Eigendecompose the scaled precision matrices
    I_psi = np.eye(T_psi.shape[0])
    J_psi = np.ones(T_psi.shape)
    K_psi = (2*p-1)*I_psi + p*(J_psi-I_psi)
    T_psi_scaled = T_psi * K_psi
    
    # theory
    T_psi_scaled = scale_diagonals_to_1(T_psi_scaled)
    
    I_theta = np.eye(T_theta.shape[0])
    J_theta = np.ones(T_theta.shape)
    K_theta = (2*n-1)*I_theta + n*(J_theta-I_theta)
    T_theta_scaled = T_theta * K_theta
    
     # theory
    T_theta_scaled = scale_diagonals_to_1(T_theta_scaled)
    
    U: "Eigenvectors of Psi"
    V: "Eigenvectors of Theta"
    _u, U = np.linalg.eig(T_psi_scaled)
    _v, V = np.linalg.eig(T_theta_scaled)
    
    if true_U is not None:
        #U = true_U
        # scales to true_U scale
        E = np.diag(np.sign(true_U)*np.sqrt(np.diag(np.abs(true_U))))
        U = E @ U @ E
    if true_V is not None:
        #V = true_V
        E = np.diag(np.sign(true_V)*np.sqrt(np.diag(np.abs(true_V))))
        V = E @ V @ E
    
    assert U.shape == (n, n)
    assert V.shape == (p, p)
    
    # We can check empirically that our calculation was correct
    # This compares lines [1] and [4] of the proof
    assert np.isclose(
        T_psi - 1 / (2*p) * T_psi * I_psi,
        1/p * (T_psi * K_psi) - 1/(2*p) * (T_psi * K_psi) * I_psi
    ).all()
    assert np.isclose(
        T_theta - 1 / (2*n) * T_theta * I_theta,
        1/n * (T_theta * K_theta) - 1/(2*n) * (T_theta * K_theta) * I_theta
    ).all()
    
    # Step three:
    # Calculate the (m, n, p) Xs tensor
    # This matrix is the 'standardized' form of the Ys input,
    # and by that we mean that it can be thought of as having
    # been drawn from a Kronecker Sum Distribution whose
    # precision matrix is the kronecker sum of the eigenvalues
    Xs: "(m, n, p)" = np.einsum("an, mnp, pb -> mab", U.T, Ys, V)
    assert Xs.shape == (m, n, p)
    
    # To check this did what we expect, let's undo the operation
    # on one of the Xs
    assert np.isclose(
        U @ Xs[0] @ V.T,
        Ys[0]
    ).all()
    
    # We can also derive that X@X.T == U.T@Y@Y.T@U
    assert np.isclose(
        np.einsum("mnp, mlp -> nl", Xs, Xs) / (correction*n),
        U.T @ T_psi @ U
    ).all()
    
    # Step four:
    # Calculate the reciprocals of the diagonals elements
    # of the Sigma matrices, which are the covariance
    # matrices for the distribution of the rows of X
    """
    a: "(n*p,)" = np.empty((n*p,))
    for idx in range(n*p):
        # for loops are inefficient, this can be optimized
        i = idx % n
        j = idx // n
        xs_to_sum = Xs[:, i, j]
        a[i + j*n] = m / (xs_to_sum * xs_to_sum).sum()
    assert a.shape == (n*p,)
    """
    # Sigma^i_j in proof is Sigma[i, j] here
    Sigma: "(n, p)" = np.empty((n, p))
    for i in range(n):
        for j in range(p):
            # For loops are bad, can be made more efficient
            xs_to_sum = Xs[:, i, j]
            Sigma[i, j] = m / (xs_to_sum * xs_to_sum).sum()
    assert Sigma.shape == (n, p)
    
    a = Sigma.T.reshape((n*p,))
    assert a.shape == (n*p, )
    if n > 1 and p > 2:
        # Check that `a[i + j*n] == Sigma[i, j]`
        assert a[1 + 2*n] == m / (Xs[:, 1, 2] * Xs[:, 1, 2]).sum()
        
    # Step five:
    # Construct the B matrix that relates `a` to the eigenvalues
    # Note that this matrix is overdetermined. Its size np by n+p
    # and hence has n^2p+np^2 total elements - however with a
    # smart argument we should be able to reduce this b/c its
    # overdetermined.  Should be reducible to (n+p)^2 elements,
    # which does add a O(np) term to the space complexity, but
    # this is subsumed by X's O(mnp) and doesn't really matter
    # because of the input's O(mnp) anyways.
    B: "(n*p, n+p)" = np.empty((n*p, n+p))
    for row in range(n*p):
        i = row % n
        j = row // n
        B[row, :] = 0
        B[row, i] = 1
        B[row, n+j] = 1
    assert B.shape == (n*p, n+p)
    
    # Step six:
    # Solve the system of linear equations to get the eigenvalues
    Ls: "Stacked eigenvalues" = np.linalg.lstsq(B, a, rcond=None)[0]
    assert Ls.shape == (n+p,)
        
    # We can use these eigenvalues to double-check the maximum
    # likelihood criteria
    D = np.linalg.inv(np.diag(kron_sum_diag(Ls[:n], Ls[n:])))
    trpD = tr_p(D, p=p)
    """
    print(U @ np.diag(trpD) @ U.T)
    print(T_psi_scaled)
    # It's not an MLE estimate unless this is true
    assert np.isclose(
        U @ np.diag(trpD) @ U.T,
        T_psi_scaled
    ).all()
    """
        
    # It seems if you're given random eigenvalues:
    Ls[:n] = np.random.random((n,))
    Ls[n:] = np.random.random((p,))
    # it still does 'well' if you have true U, V
        
    # Step seven:
    # Calculate Psi, Theta, finally!
    # (Scaling to 1 is important for LASSO as then all columns
    # will be on the same scale)
    Psi = scale_diagonals_to_1(U @ np.diag(Ls[:n]) @ U.T)
    Theta = scale_diagonals_to_1(V @ np.diag(Ls[n:]) @ V.T)
    assert Psi.shape == (n, n)
    assert Theta.shape == (p, p)
    
    # Step eight:
    # Perform Lasso regression
    # (Could replace by row-wise Lasso regression
    # as in the original algorithm)
    if beta_1 > 0:
        Psi = LASSO(np.eye(Psi.shape[0]), Psi-I_psi, beta_1 / n) + I_psi
    if beta_2 > 0:
        Theta = LASSO(np.eye(Theta.shape[0]), Theta-I_theta, beta_2 / p) + I_theta
    
    return Psi, Theta, (U, V)

In [104]:
u, U = np.linalg.eig(Psi_gen)
v, V = np.linalg.eig(Theta_gen)
Psi, Theta, extras = analyticBiGLasso(
    Ys,
    0,#0.02,
    0,#0.02,
    true_U=U,
    true_V=V
)

# Let's make some assertions to check the proof
W = np.linalg.inv(kron_sum(Psi_gen, Theta_gen))
trpW = tr_p(W, p=p)
print(trpW)
D = np.linalg.inv(np.diag(kron_sum_diag(u, v)))
I = np.eye(p)
trpD = tr_p(D, p=p)
What = (kron_prod(U, I) @ D @ kron_prod(U.T, I))
trpWhat = tr_p(What, p=p)

# These assertions verify that the trp[W] reduction works
# i.e. we can get the eigenvectors!
assert np.isclose(
    trpW,
    trpWhat
).all()
assert np.isclose(
    trpW,
    U @ trpD @ U.T
).all()

# Efficiency
print("===Psi===")
print(Psi_cm := generate_confusion_matrices(Psi, Psi_gen, mode='Negative'))
print(
    f"{precision(Psi_cm)=:.3f}",
    f"\n{recall(Psi_cm)=:.3f}",
    f"\n{accuracy(Psi_cm)=:.3f}"
)
print("\n==Theta==")
print(Theta_cm := generate_confusion_matrices(Theta, Theta_gen, mode='Negative'))
print(
    f"{precision(Theta_cm)=:.3f}",
    f"\n{recall(Theta_cm)=:.3f}",
    f"\n{accuracy(Theta_cm)=:.3f}"
)

AssertionError: 

In [15]:
from Scripts.scBiGLasso import scBiGLasso
Psi, Theta = scBiGLasso(
    100,
    1e-3,
    Ys,
    0.01,
    0.01,
)
print("===Psi===")
print(Psi_cm := generate_confusion_matrices(Psi, Psi_gen, mode='Negative'))
print(
    f"{precision(Psi_cm)=:.3f}",
    f"\n{recall(Psi_cm)=:.3f}",
    f"\n{accuracy(Psi_cm)=:.3f}"
)
print("\n==Theta==")
print(Theta_cm := generate_confusion_matrices(Theta, Theta_gen, mode='Negative'))
print(
    f"{precision(Theta_cm)=:.3f}",
    f"\n{recall(Theta_cm)=:.3f}",
    f"\n{accuracy(Theta_cm)=:.3f}"
)

===Psi===
[[  72.  810.]
 [ 124. 1544.]]
precision(Psi_cm)=0.082 
recall(Psi_cm)=0.367 
accuracy(Psi_cm)=0.634

==Theta==
[[  72.  792.]
 [ 146. 1342.]]
precision(Theta_cm)=0.083 
recall(Theta_cm)=0.330 
accuracy(Theta_cm)=0.601


In [460]:
nibig = niBiGLasso()
T_psi, T_theta = nibig.get_empiricals(Ys)
nibig.fit(T_psi, T_theta)
Psi, Theta = nibig.shrink(0.02, 0.02)#0.02, 0.02
nibig.print_vindication()
print("===Psi===")
print(Psi_cm := generate_confusion_matrices(Psi, Psi_gen, mode='Negative'))
print(
    f"{precision(Psi_cm)=:.3f}",
    f"\n{recall(Psi_cm)=:.3f}",
    f"\n{accuracy(Psi_cm)=:.3f}"
)
print("\n==Theta==")
print(Theta_cm := generate_confusion_matrices(Theta, Theta_gen, mode='Negative'))
print(
    f"{precision(Theta_cm)=:.3f}",
    f"\n{recall(Theta_cm)=:.3f}",
    f"\n{accuracy(Theta_cm)=:.3f}"
)

Psi vindication: 0.9997221875594935
Theta vindication: 0.9999964625115909
===Psi===
[[   4.  184.]
 [ 206. 2156.]]
precision(Psi_cm)=0.021 
recall(Psi_cm)=0.019 
accuracy(Psi_cm)=0.847

==Theta==
[[   0.    6.]
 [ 280. 2066.]]
precision(Theta_cm)=0.000 
recall(Theta_cm)=0.000 
accuracy(Theta_cm)=0.878


# Empirical Verification of Problem

In [162]:
# Empirically verify Theorem 1
# (what are the chances the equations would work out
# for randomly selected matrices if it were false?)
T = np.random.random((2, 2))
I = np.eye(2)
J = np.ones((2, 2))
K = (2*p-1)*I + p*(J-I)
p = 2
trpW = T * K

# Verify lines 1-4, these should be equal
print(T - 1/(2*p) * T * I)
print(1/p * trpW - 1/(2*p)*trpW * I)
print(np.isclose(T - 1/(2*p) * T * I, 1/p * trpW - 1/(2*p)*trpW * I))

from Scripts.utilities import tr_p, kron_prod
U = np.random.random((3, 3))
D = np.random.random((6, 6))

# Verify lines 6-7, these should be equal
print(tr_p(kron_prod(U, I) @ D @ kron_prod(U.T, I), p=2))
print(U @ tr_p(D, p=2) @ U.T)
print(
    np.isclose(
        tr_p(kron_prod(U, I) @ D @ kron_prod(U.T, I), p=2),
        U @ tr_p(D, p=2) @ U.T
    )
)

[[0.43  0.132]
 [0.014 0.028]]
[[0.43  0.132]
 [0.014 0.028]]
[[ True  True]
 [ True  True]]
[[0.819 1.977 0.627]
 [2.15  4.971 1.566]
 [0.729 1.693 0.531]]
[[0.819 1.977 0.627]
 [2.15  4.971 1.566]
 [0.729 1.693 0.531]]
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


In [199]:
from Scripts.generate_data import generate_sparse_posdef_matrix
from Scripts.utilities import kron_sum, kron_sum_diag
# Verify lemma 1

# First the |Psi kronsum Theta| equality
Psi = generate_sparse_posdef_matrix(4, 16)[0]
Theta = generate_sparse_posdef_matrix(5, 25)[0]
u, U = np.linalg.eigh(Psi)
v, V = np.linalg.eigh(Theta)
kr_diag = np.diag(kron_sum_diag(u, v))
print(np.linalg.det(kron_sum(Psi, Theta)))
print(np.linalg.det(kr_diag))
print(np.isclose(np.linalg.det(kr_diag), np.linalg.det(kron_sum(Psi, Theta))))

# Verify the first trace equality
Y = np.random.random((4, 5))
X = U.T @ Y @ V
print(np.trace(Psi @ Y @ Y.T))
print(np.trace(np.diag(u) @ X @ X.T))
print(np.isclose(np.trace(Psi @ Y @ Y.T), np.trace(np.diag(u) @ X @ X.T)))

# Verify the second trace equality
print(np.trace(Theta @ Y.T @ Y))
print(np.trace(np.diag(v) @ X.T @ X))
print(np.isclose(np.trace(Theta @ Y.T @ Y), np.trace(np.diag(v) @ X.T @ X)))

999.753915090151
999.7539150901546
True
7.299938075612053
7.29993807561205
True
1.5968853534425984
1.5968853534425982
True


In [140]:
# How good is the MLE of U, really?
errs = []
tries = 1000
for i in range(tries):
    Psi_gen, Theta_gen, Ys = generate_Ys(
        m=(m:=10),
        n=(n:=21),
        p=(p:=19),
        expected_nonzero_psi=n**2 / 5, # (divide by 5 for sparsity)
        expected_nonzero_theta=p**2 / 5,
        structure="Kronecker Sum",
        posdef_distr=invwishart
    )
    correction = m#-1
    T: "(n, n)" = np.einsum("mnp, mlp -> nl", Ys, Ys) / (correction * n)
    u, U = np.linalg.eig(Psi_gen)
    v, V = np.linalg.eig(Theta_gen)
    D = np.linalg.inv(np.diag(kron_sum_diag(u, v)))
    trpD = tr_p(D, p=p)
    #W = np.linalg.inv(kron_sum(Psi_gen, Theta_gen))
    #trpW = tr_p(W, p=p)
    I = np.eye(U.shape[0])
    J = np.ones(U.shape)
    K = (2*p-1)*I + p*(J-I)
    #print(np.linalg.norm(trpW - T * K, ord='fro'))
    diff = U @ trpD @ U.T - T * K
    #diff = scale_diagonals_to_1(U @ trpD @ U.T) - scale_diagonals_to_1(T * K)
    errs.append(np.linalg.norm(diff, ord='fro'))
sum(errs) / tries

189.74988738109164